# KPPS H步预测误差方差分解的Python实现

First: Jun 20, 2024

```{contents}
```

## KPPS

KPPS H-step-ahead forecast error variance decompositions:

$$
\theta_{ij}^g(H)=\frac{\sigma_{jj}^{-1}\sum_{h=0}^{H-1}(e_i^{\prime}A_h\Sigma e_j)^2}{\sum_{h=0}^{H-1}(e_i^{\prime}A_h\Sigma A_h^{\prime}e_i)}
$$

为了实现这个FEVD公式，我们需要先理解每个符号的含义和计算步骤：

- $ \theta_{ij}^g(H) $ 是广义预测误差方差分解（GFEVD）。
- $ \sigma_{jj}^{-1} $ 是残差协方差矩阵 $\Sigma$ 的逆矩阵中的对角元素。
- $ A_h $ 是VAR模型的冲击响应矩阵（Impulse Response Matrix）在第 $ h $ 步。
- $ e_i $ 和 $ e_j $ 是单位向量，用于选择对应的变量。

计算步骤：

1. 拟合VAR模型，得到模型参数。
2. 计算冲击响应矩阵 $ A_h $。
3. 根据公式计算每个变量的GFEVD。



本文使用R语言内置的数据集中名为Canada的数据集。该数据集是时间序列数据，包含了加拿大的年度未调整生产、储蓄和投资数据，时间跨度从1937年到1965年。
- Year - 年份
- Production - 生产总值
- Savings - 储蓄总值
- Investment - 投资总值

In [7]:
import pandas as pd

import matplotlib.pyplot as plt

# 读取CSV文件
df = pd.read_csv('Canada_data.csv')

# 删除任何可能的空行或列（如果有）
df.dropna(inplace=True)

# 检查数据集的前几行
print(df.head())

            e        prod          rw     U
0  929.610514  405.366466  386.136109  7.53
1  929.803985  404.639834  388.135759  7.70
2  930.318388  403.814883  390.540113  7.47
3  931.427687  404.215773  393.963817  7.27
4  932.662006  405.046714  396.764691  7.37


## VAR模型

In [8]:
from statsmodels.tsa.api import VAR

model = VAR(df)
results = model.fit(maxlags=2, ic='aic')  # 可以根据需要调整滞后阶数

获取模型残差的协方差矩阵及其逆矩阵

In [9]:
import numpy as np
sigma_u = results.sigma_u

## 脉冲响应函数（IRF）

计算在第 $ h$ 步的冲击响应矩阵 $ A_h$


In [10]:
H=5
irf = results.irf(H)  # 设置脉冲响应函数的预测步数，例如5步
irf_non_orth = irf.irfs  # orth_irfs 为正交化IRF，irfs 为非正交化IRF
irf_non_orth # h, j, i

array([[[ 1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 1.6378206 ,  0.16727167, -0.06311863,  0.26558478],
        [-0.17276581,  1.1504282 ,  0.0513039 , -0.47850131],
        [-0.26883287, -0.081065  ,  0.89547833,  0.01213003],
        [-0.58076382, -0.07811707,  0.01866214,  0.6189315 ]],

       [[ 2.01915006,  0.34911497, -0.1425158 ,  0.65124297],
        [ 0.16764893,  1.15539453, -0.01191318,  0.12401542],
        [-0.30622451, -0.21694805,  0.86759379, -0.14194662],
        [-0.89234278, -0.1847587 ,  0.10271259,  0.19527081]],

       [[ 2.24263533,  0.51890237, -0.2308082 ,  1.14696461],
        [ 0.35801917,  1.14252986, -0.11435232,  0.74159876],
        [-0.17807312, -0.32275268,  0.83873951, -0.28809984],
        [-1.05145989, -0.2807327 ,  0.17637234, -0.22933386]],



## 基于MA过程

In [11]:
MA_A = results.ma_rep(maxn=5)
MA_A

array([[[ 1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 1.6378206 ,  0.16727167, -0.06311863,  0.26558478],
        [-0.17276581,  1.1504282 ,  0.0513039 , -0.47850131],
        [-0.26883287, -0.081065  ,  0.89547833,  0.01213003],
        [-0.58076382, -0.07811707,  0.01866214,  0.6189315 ]],

       [[ 2.01915006,  0.34911497, -0.1425158 ,  0.65124297],
        [ 0.16764893,  1.15539453, -0.01191318,  0.12401542],
        [-0.30622451, -0.21694805,  0.86759379, -0.14194662],
        [-0.89234278, -0.1847587 ,  0.10271259,  0.19527081]],

       [[ 2.24263533,  0.51890237, -0.2308082 ,  1.14696461],
        [ 0.35801917,  1.14252986, -0.11435232,  0.74159876],
        [-0.17807312, -0.32275268,  0.83873951, -0.28809984],
        [-1.05145989, -0.2807327 ,  0.17637234, -0.22933386]],





## 计算广义预测误差方差分解（GFEVD）
函数 `compute_gfevd` 实现公式中所述的计算，遍历每个变量  $i$  和  $j$ ，并根据给定的预测步数  $H$  进行计算。

In [12]:
import numpy as np

# 计算 GFEVD
def compute_gfevd(H, A, sigma_u):
    n_vars = A.shape[1]
    gfevd = np.zeros((n_vars, n_vars))
    e = np.eye(n_vars)
    
    for i in range(n_vars):
        for j in range(n_vars):
            num = 1/sigma_u[j, j] * np.sum([(e[i].T @ A[h] @ sigma_u @ e[j]) ** 2 for h in range(H)]) 
            den = np.sum([(e[i].T @ A[h] @ sigma_u @ A[h].T @ e[i]) for h in range(H)])
            gfevd[i, j] = num / den
    return gfevd

In [7]:
# 计算 GFEVD
H=5
gfevd = compute_gfevd(H, irf_non_orth, sigma_u, sigma_u_inv)

gfevd = pd.DataFrame(gfevd[:,:], index=df.columns,
             columns=df.columns)


gfevd

,e,prod,rw,U
e,1.314613,0.154236,0.070726,0.300967
prod,0.005640,0.976393,0.011240,0.071030
rw,0.074124,0.027349,0.970155,0.052102
U,1.289732,0.115564,0.119263,0.806846


In [8]:
# 计算 GFEVD
gfevd = compute_gfevd(H, MA_A, sigma_u, sigma_u_inv)

gfevd = pd.DataFrame(gfevd[:,:], index=df.columns,
             columns=df.columns)


gfevd

,e,prod,rw,U
e,1.314613,0.154236,0.070726,0.300967
prod,0.005640,0.976393,0.011240,0.071030
rw,0.074124,0.027349,0.970155,0.052102
U,1.289732,0.115564,0.119263,0.806846


两者结果是一致的。


## 归一化和结果展示
归一化计算结果，确保分母为1。

通过这种方式，你可以计算并查看不同变量在给定预测步数内的广义预测误差方差分解（GFEVD）结果。这种方法不仅考虑了直接的冲击影响，还考虑了残差协方差矩阵的逆，从而提供了更全面的分析。

In [13]:
def scale_one(df):
    # j 归一化
    s1 = df.values
    s2 = df.values.sum(axis=1)
    s3 = (s1.T/s2).T
    return pd.DataFrame(s3, index=df.index, columns=df.index)

In [14]:
scale_one(gfevd)

,e,prod,rw,U
e,0.714253,0.083799,0.038427,0.163521
prod,0.005299,0.917401,0.010561,0.066738
rw,0.065963,0.024338,0.863334,0.046365
U,0.553199,0.049569,0.051155,0.346077


In [15]:
scale_one(gfevd).sum(axis=1)

e       1.0
prod    1.0
rw      1.0
U       1.0
dtype: float64